# Hyper parameters optimization #

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [1]:
n_folds = 5 # this sets global setting of which how many bootstraps to use
n_hold_out = 1
repeats = 2
n_trials = 10
#first round of optimization
lgb_params = {"bagging_fraction": 0.9522534844058304, 
              "boosting_type": "dart", 
              "objective": "regression",
              "feature_fraction": 0.42236910941558053, 
              "lambda_l1": 0.020847266580277746, 
              "lambda_l2": 2.8448564854773326, 
              "learning_rate": 0.11484015430016059, 
              "max_depth": 3, 
              "max_leaves": 35, 
              "min_data_in_leaf": 9,
              "num_iterations": 150
             }
debug_local = True #to use local version

In [2]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import TupleWith, Repeat, Collect
from yspecies.config import *
from yspecies.preprocess import FeatureSelection, DataExtractor
from yspecies.partition import DataPartitioner, PartitionParameters
from yspecies.selection import ShapSelector
from yspecies.tuning import Tune
from yspecies.models import ResultsCV, CrossValidator
from yspecies.results import FeatureSummary
import optuna
from optuna import Study, Trial

In [5]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: TkAgg


### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [6]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [7]:
data = ExpressionDataset.from_folder(locations.interim.selected)
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(408, 12323)",12323,38,408,"(12323, 2)","(38, 19)"


## Setting up ShapSelector ##

Deciding on selection parameters (which fields to include, exclude, predict)

In [8]:
partition_params = PartitionParameters(n_folds, n_hold_out, 2, [],  42)

## Setting up features to select ##

In [9]:
selection = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = "lifespan", #column to predict
    categorical = ["tissue"],
    select_by = "gain",
    importance_type = "gain",
    clean_y_na = True
)

In [10]:
select_lifespan = selection

In [11]:
url = f'sqlite:///' +str((locations.metrics.lifespan / "study.sqlite").absolute())
print('loading (if exists) study from '+url)
storage = optuna.storages.RDBStorage(
    url=url
    #engine_kwargs={'check_same_thread': False}
)

study = optuna.multi_objective.study.create_study(directions=['maximize', 'maximize'], storage = storage)

loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/metrics/lifespan/study.sqlite


create_study is experimental (supported from v1.4.0). The interface can change in the future.
NSGAIIMultiObjectiveSampler is experimental (supported from v1.5.0). The interface can change in the future.
RandomMultiObjectiveSampler is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-08-16 13:02:41,226] A new study created with name: no-name-d458158c-12be-409a-b3a0-73a3fbe9e089
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.


In [20]:
def objective_parameters(trial: Trial) -> dict:
    return {
        'objective': 'regression',
        'metric': {'mae', 'mse', 'huber'},
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting_type', ['dart', 'gbdt']),
        'lambda_l1': trial.suggest_uniform('lambda_l1', 0.01, 4.0),
        'lambda_l2': trial.suggest_uniform('lambda_l2', 0.01, 4.0),
        'max_leaves': trial.suggest_int("max_leaves", 15, 25),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.3, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 1.0),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 3, 8),
        'drop_rate': trial.suggest_uniform('drop_rate', 0.1, 0.3),
        "verbose": -1
    }
optimization_parameters = objective_parameters

In [21]:
from yspecies.workflow import SplitReduce

def side(i: int):
    print(i)
    return i

prepare_partition = SplitReduce(
    outputs = DataPartitioner(), 
    split = lambda x: [(x[0], replace(partition_params, seed=side(x[2])))], 
    reduce = lambda x, output: (output[0], x[1]) 
)                               
partition_and_cv = Pipeline(
    [
        ("prepare partition", prepare_partition),
        ("shap_computation", ShapSelector()) #('crossvalidator', CrossValidator())        
    ]
)

partition_and_cv_repeat =  Pipeline([
    ("repeat_cv_pipe", Repeat(partition_and_cv, repeats, lambda x, i: [x[0], x[1], i] )),
    #("collect_mean", Collect(fold=lambda results: np.array([r.last("huber") for r in results]).mean()))
    ("collect_mean", Collect(fold=lambda results: FeatureSummary(results).average_metrics.R2, FeatureSummary(results).average_metrics.R2))    
    ]
    )

p = Pipeline([
     ('extractor', DataExtractor()),
     ('tune', Tune(partition_and_cv_repeat, study = study, n_trials = n_trials, parameters_space = optimization_parameters))    
])

In [22]:
best = p.fit_transform((data, select_lifespan))
best

0
Training until validation scores don't improve for 10 rounds
[150]	valid_0's huber: 20.132	valid_0's l2: 2218.95	valid_0's l1: 22.6971
Did not meet early stopping. Best iteration is:
[150]	valid_0's huber: 20.132	valid_0's l2: 2218.95	valid_0's l1: 22.6971
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[74]	valid_0's huber: 3.11956	valid_0's l2: 58.0973	valid_0's l1: 3.79165
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[93]	valid_0's huber: 2.30362	valid_0's l2: 27.3022	valid_0's l1: 2.90216
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[89]	valid_0's huber: 1.98584	valid_0's l2: 15.6843	valid_0's l1: 2.55455


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.


1
Training until validation scores don't improve for 10 rounds
[150]	valid_0's huber: 1.8195	valid_0's l2: 20.0327	valid_0's l1: 2.38288
Did not meet early stopping. Best iteration is:
[150]	valid_0's huber: 1.8195	valid_0's l2: 20.0327	valid_0's l1: 2.38288
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[63]	valid_0's huber: 3.04317	valid_0's l2: 53.5174	valid_0's l1: 3.71913
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[108]	valid_0's huber: 2.13521	valid_0's l2: 25.9303	valid_0's l1: 2.70193
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[102]	valid_0's huber: 19.0372	valid_0's l2: 1985.18	valid_0's l1: 21.4931


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
[W 2020-08-16 10:55:17,146] Trial 0 failed because of the following error: NameError("name 'FeatureSummary' is not defined")
Traceback (most recent call last):
  File "/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/optuna/study.py", line 709, in _run_trial
    result = func(trial)
  File "../yspecies/tuning.py", line 107, in objective
    result = self.transformer.fit_transform((data, params))
  File "/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/sklearn/pipeline.py", line 376, in fit_transform
    return last_step.fit_transform(Xt, y, **fit_params_last_step)
  File "/opt/miniconda3/envs/yspecies/lib/pytho

NameError: name 'FeatureSummary' is not defined

In [ ]:
best["metric"] =  ["mae", "mse", "huber"]
best['objective'] = 'regression'
best

## Getting shap results with the best parameters ##

In [ ]:
def make_shap(params: dict):
    partition_shap_pipe = Pipeline([
        ("partitioner", DataPartitioner()),
        ('prepare_for_partitioning', TupleWith(lgb_params)),
        ("shap_computation", ShapSelector())
    ]
    )
    repeated_cv =  Repeat(partition_shap_pipe, repeats, lambda x,i: (x[0], replace(x[1], seed = i)))
    return Pipeline([
        ('extractor', DataExtractor()),
        ('prepare_for_partitioning', TupleWith(params)), # to extract the data required for ML from the dataset
        ("partition_shap", repeated_cv)]
        )